# Tracing for LangChain (Python)

This notebook works well with the `Data Science 2.0 kernel` on a SageMaker Studio `ml.t3.medium` instance.

### Install required packages

In [ ]:
%%capture --no-stderr

!pip install -U langfuse==2.39.3
!pip install -U langchain==0.2.11
!pip install -U langchain-community==0.2.10
!pip install -U langchain-aws==0.1.12

In [ ]:
!pip freeze | grep -E "langchain|langfuse"

langchain==0.2.11
langchain-aws==0.1.12
langchain-community==0.2.10
langchain-core==0.2.24
langchain-text-splitters==0.2.2
langfuse==2.39.3


### Environment Setup and LangFuse Initialization

In [ ]:
import os
from langfuse.callback import CallbackHandler

# Get the API key from the LangFuse project settings and set it as an environment variable
os.environ["LANGFUSE_PUBLIC_KEY"] = 'pk-lf-***'
os.environ["LANGFUSE_SECRET_KEY"] = 'sk-lf-***'
os.environ["LANGFUSE_HOST"] = "http://langfu-***-****.{region}.elb.amazonaws.com"

# Initialize LangFuse Callback Handler
langfuse_handler = CallbackHandler()

In [ ]:
# connection test
langfuse_handler.auth_check()

True

In [ ]:
import boto3
from langchain_aws import ChatBedrock as BedrockChat

region = boto3.Session().region_name

model_kwargs={
    "max_tokens": 512,
    "temperature": 0,
    "top_p": 0.9
}

model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

llm = BedrockChat(model_id=model_id, region_name=region, model_kwargs=model_kwargs)
llm

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from operator import itemgetter

prompt1 = ChatPromptTemplate.from_template("What is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "What country is the city {city} in? Respond in {language}"
)

chain1 = prompt1 | llm | StrOutputParser()
chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | llm
    | StrOutputParser()
)

In [ ]:
# Invoke chain with LangFuse callback handler
result = chain2.invoke(
    {"person": "Obama", "language": "Spanish"},
    config={"callbacks": [langfuse_handler]}
)

In [ ]:
print(result)

La ciudad de Honolulu, donde nació Barack Obama el 4 de agosto de 1961, está ubicada en Hawái, un estado de los Estados Unidos.

Aunque pasó gran parte de su infancia en Indonesia y vivió en varios otros lugares como Los Ángeles, Nueva York, Boston y Chicago antes de convertirse en presidente, Chicago se considera su base política principal. Allí trabajó como organizador comunitario, enseñó derecho constitucional en la Universidad de Chicago e inició su carrera política sirviendo en el Senado estatal de Illinois y luego en el Senado de los Estados Unidos representando a Illinois antes de ser elegido como el 44º presidente de los Estados Unidos en 2008.

Si bien nació en Honolulu, Hawái, las conexiones de Barack Obama con Chicago como adulto fueron muy sólidas e integrales para su ascenso en la política antes de convertirse en presidente.


## References

- [Langfuse Documents](https://langfuse.com/docs)
    - [Tracing for Langchain (Python & JS/TS)](https://langfuse.com/docs/integrations/langchain/tracing)